### Import Dependencies

In [1]:
import openai
import instructor
from qdrant_client import QdrantClient

from pydantic import BaseModel, Field

### Mock Example

In [2]:
prompt = """
You are a helpful assistant.
Return an answer to the question.
Question: What is your name?
"""

In [3]:
openai_response = openai.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": prompt}
    ],
    temperature=0
)

print(openai_response.choices[0].message.content)

I am ChatGPT, your AI assistant. How can I help you today?


In [5]:
openai_response

ChatCompletion(id='chatcmpl-D0r1EuVrJrzXBWiSiS6MlFEuPH2Rv', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I am ChatGPT, your AI assistant. How can I help you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1769096176, model='gpt-4.1-mini-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_376a7ccef1', usage=CompletionUsage(completion_tokens=16, prompt_tokens=28, total_tokens=44, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

### Add Instructor (Structured Outputs)

In [4]:
client = instructor.from_openai(openai.OpenAI())

In [6]:
class RAGGenerationResponse(BaseModel):
    answer: str = Field(description="The answer to the question")

In [7]:
response, raw_response = client.chat.completions.create_with_completion(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": prompt}
    ],
    temperature=0,
    response_model=RAGGenerationResponse
)

In [8]:
response

RAGGenerationResponse(answer='My name is ChatGPT. How can I assist you today?')

In [9]:
raw_response

ChatCompletion(id='chatcmpl-D0r1NqVdWwWpepqtNBgPEBqHR3Uqv', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_MySYSSnY7xYYfnOuc1GQQIWg', function=Function(arguments='{"answer":"My name is ChatGPT. How can I assist you today?"}', name='RAGGenerationResponse'), type='function')]))], created=1769096185, model='gpt-4.1-mini-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_376a7ccef1', usage=CompletionUsage(completion_tokens=17, prompt_tokens=95, total_tokens=112, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=None, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=None), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [10]:
class RAGGenerationResponse(BaseModel):
    answer: str = Field(description="The answer to the question")
    reasoning: str = Field(description="The reasoning for the answer")

In [11]:
response, raw_response = client.chat.completions.create_with_completion(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": prompt}
    ],
    temperature=0,
    response_model=RAGGenerationResponse
)

In [12]:
response

RAGGenerationResponse(answer='My name is ChatGPT.', reasoning='I am an AI language model created by OpenAI, and I am commonly known as ChatGPT.')

### RAG Example

In [13]:
class RAGGenerationResponse(BaseModel):
    answer: str = Field(description="The answer to the question")

In [14]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )

    return response.data[0].embedding


def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }


def process_context(context):

    formatted_context = ""

    for id, chunk, rating in zip(context["retrieved_context_ids"], context["retrieved_context"], context["retrieved_context_ratings"]):
        formatted_context += f"- ID: {id}, rating: {rating}, description: {chunk}\n"

    return formatted_context


def build_prompt(preprocessed_context, question):

    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt


def generate_answer(prompt):

    response, raw_response = client.chat.completions.create_with_completion(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": prompt}],
        temperature=0,
        response_model=RAGGenerationResponse
    )

    return response


def rag_pipeline(question, qdrant_client, top_k=5):

    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    final_result = {
        "datamodel": answer,
        "answer": answer.answer,
        "question": question,
        "retrieved_context_ids": retrieved_context["retrieved_context_ids"],
        "retrieved_context": retrieved_context["retrieved_context"],
        "similarity_scores": retrieved_context["similarity_scores"]
    }

    return final_result

In [15]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [16]:
output = rag_pipeline("Can I get a charging cable? Please suggest me a good one.", qdrant_client)

In [17]:
output

{'datamodel': RAGGenerationResponse(answer='You can get a good charging cable from the available products. The YOXXZUS Switch Charger (ID: B0BWQYJY87) comes with a high-quality 5 feet Type-C integrated cable, which is safe, stable, and designed for Switch OLED/Switch Lite. It offers fast charging and has multiple protections like short circuit, over current, surge, and overload protection.\n\nIf you are looking for a charging cable specifically for Oculus Quest 2 controllers, the HiBloks Magnetic Controllers Charging Kit (ID: B09PVCVQDP) includes a magnetic charging cable along with rechargeable batteries and battery covers, making charging effortless and convenient.\n\nSo, depending on your device, these two options are good choices for charging cables.'),
 'answer': 'You can get a good charging cable from the available products. The YOXXZUS Switch Charger (ID: B0BWQYJY87) comes with a high-quality 5 feet Type-C integrated cable, which is safe, stable, and designed for Switch OLED/Swi

In [18]:
print(output["answer"])

You can get a good charging cable from the available products. The YOXXZUS Switch Charger (ID: B0BWQYJY87) comes with a high-quality 5 feet Type-C integrated cable, which is safe, stable, and designed for Switch OLED/Switch Lite. It offers fast charging and has multiple protections like short circuit, over current, surge, and overload protection.

If you are looking for a charging cable specifically for Oculus Quest 2 controllers, the HiBloks Magnetic Controllers Charging Kit (ID: B09PVCVQDP) includes a magnetic charging cable along with rechargeable batteries and battery covers, making charging effortless and convenient.

So, depending on your device, these two options are good choices for charging cables.
